In [33]:
import pandas as pd
import seaborn as sns
import numpy as np
from pprint import pprint

pd.set_option("max_rows", 10)

dta = pd.read_csv("data/network_traffic.csv")

dta

,date,l_ipn,r_asn,f
0,2006-07-01,0,701,1
1,2006-07-01,0,714,1
2,2006-07-01,0,1239,1
3,2006-07-01,0,1680,1
4,2006-07-01,0,2514,1
...,...,...,...,...
20798,2006-09-30,8,36351,3
20799,2006-09-30,8,36856,4
20800,2006-09-30,9,2152,1
20801,2006-09-30,9,8070,1


In [40]:
ary = []

for i in range(10):
    ary.append(dta[dta['l_ipn'] == i])

In [41]:
ary[0].describe()

,l_ipn,r_asn,f
count,3980.0,3980.000000,3980.000000
mean,0.0,11890.358040,26.426382
std,0.0,9104.685237,232.733282
min,0.0,4.000000,1.000000
25%,0.0,4808.000000,1.000000
50%,0.0,8708.000000,1.000000
75%,0.0,17228.000000,2.000000
max,0.0,39834.000000,5059.000000


In [46]:
stds = []
for frame in ary:
    stds.append(frame.f.std())

stds1 = pd.DataFrame(stds)
stds1.describe()

,0
count,10.000000
mean,2450.910462
std,7209.307911
min,15.580845
25%,27.979783
50%,170.483057
75%,331.079349
max,22963.981816
